In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/ME')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /ME


In [65]:
!pip3 install https://download.pytorch.org/whl/cu80/torch-1.0.0-cp36-cp36m-linux_x86_64.whl

     |████████████████████████████████| 532.5MB 29kB/s 
ERROR: torchvision 0.7.0+cu101 has requirement torch==1.6.0, but you'll have torch 1.0.0 which is incompatible.
  Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101


In [23]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.linear1 = torch.nn.Linear(4383, 3125)
    # self.linearTemp = torch.nn.Linear(125, 20)
    # self.linearTemp2 = torch.nn.Linear(20, 1)
    self.linear2 = torch.nn.Linear(3125, 625)
    self.linear3 = torch.nn.Linear(625, 125)
    self.linear4 = torch.nn.Linear(125, 25)
    self.linear5 = torch.nn.Linear(25, 5)
    self.linear6 = torch.nn.Linear(5, 1)

  def forward(self, x):
    x = F.dropout(x, training=self.training)
    x = F.softplus(F.dropout(self.linear1(x), training=self.training))
    # x = F.softplus(F.dropout(self.linearTemp(x), training=self.training))
    x = F.softplus(F.dropout(self.linear2(x), training=self.training))
    x = F.softplus(F.dropout(self.linear3(x), training=self.training))
    x = F.softplus(F.dropout(self.linear4(x), training=self.training))
    x = F.softplus(F.dropout(self.linear5(x), training=self.training))
    return self.linear6(x)
    # return self.linearTemp2(x)

class SimpleNet(nn.Module):
  def __init__(self):
    super(SimpleNet, self).__init__()
    self.linear = torch.nn.Linear(15, 1)
    # self.linear1 = torch.nn.Linear(15, 125)
    # self.linear2 = torch.nn.Linear(125, 25)
    # self.linear3 = torch.nn.Linear(25, 5)
    # self.linear4 = torch.nn.Linear(5, 1)

  def forward(self, x):
    return self.linear(x)
    # x = F.dropout(x, training=self.training)
    # x = F.softplus(F.dropout(self.linear1(x), training=self.training))
    # x = F.softplus(F.dropout(self.linear2(x), training=self.training))
    # x = F.softplus(F.dropout(self.linear3(x), training=self.training))
    # return self.linear4(x)

In [2]:
data1 = np.loadtxt('/ME/My Drive/Basketball_Data/data_01-04.csv', delimiter=',')
data2 = np.loadtxt('/ME/My Drive/Basketball_Data/data.csv', delimiter=',')

In [3]:
data = np.concatenate((data1, data2))
players = data[:, 20:188]
for i in range(1, 26):
  players = np.concatenate((players, data[:, 20+168*i:20+168*(i+1)]))

In [5]:
data[:, 5:20] = (data[:, 5:20] - np.mean(data[:, 5:20], axis=0))/np.std(data[:, 5:20], axis=0)
p_mean = np.mean(players, axis=0)
p_std = np.std(players, axis=0)
for i in range(0, 26):
  data[:, 20+168*i:20+168*(i+1)] = (data[:, 20+168*i:20+168*(i+1)] - p_mean) / p_std

In [6]:
testing_index = int(.9 * data.shape[0])
# np.random.shuffle(data)
validation_index = int(.8 * data.shape[0])
testing_data_set = data[testing_index:]
data = data[:testing_index]

In [7]:
np.random.shuffle(data)

In [8]:
data_training = data[:validation_index, 5:4388]
simple_training = data[:validation_index, 5:20]
labels_training = data[:validation_index, 4388]
data_val = data[validation_index:, 5:4388]
simple_val = data[validation_index:, 5:20]
labels_val = data[validation_index:, 4388]
data_test = testing_data_set[:, 5:4388]
simple_test = testing_data_set[:, 5:20]
labels_test = testing_data_set[:, 4388]
print(data.shape)
print(data_training.shape)
print(data_val.shape)

(22946, 4389)
(20396, 4383)
(2550, 4383)


In [6]:
# data_set = data[:,5:4388]
# data_set_simple = data[:,5:20]
# labels = data[:, 4388]

In [9]:
labels_training = np.where(labels_training < 0, -1, 1)
labels_val = np.where(labels_val < 0, -1, 1)
labels_test = np.where(labels_test < 0, -1, 1)

In [71]:
small_simple_training = simple_training[:100]
small_labels = labels_training[:100]
small_val = simple_training[40:80]
small_val_labels = labels_training[40:80]

In [8]:
# validation_index = int(.8 * labels.size)
# testing_index = int(.9 * labels.size)
# data_training = data_set[:validation_index]
# data_val = data_set[validation_index:testing_index]
# data_test = data_set[testing_index:]
# simple_training = data_set_simple[:validation_index]
# simple_val = data_set_simple[validation_index:testing_index]
# simple_test = data_set_simple[testing_index:]
# labels_training = labels[:validation_index]
# labels_val = labels[validation_index:testing_index]
# labels_test = labels[testing_index:]

In [82]:
import random

BATCH_SIZE = 2
EPOCH = 50

simple_net = SimpleNet()
simple_net.to(torch.device("cuda"))
optimizer = torch.optim.SGD(simple_net.parameters(), lr=0.0025)
loss_func = torch.nn.MSELoss()

In [83]:
simple_net.training = True

for i in range(1000):
  batch_start = random.randrange(small_labels.size - BATCH_SIZE)
  batch_x = torch.Tensor(small_simple_training[batch_start:batch_start+BATCH_SIZE]).to(torch.device("cuda:0"))
  batch_y = torch.Tensor(small_labels[batch_start:batch_start+BATCH_SIZE].reshape(BATCH_SIZE, 1)).to(torch.device("cuda:0"))
  prediction = simple_net(batch_x)     # input x and predict based on x

  loss = loss_func(prediction, batch_y)     # must be (1. nn output, 2. target)
  loss = loss.to(torch.device("cuda:0"))
  optimizer.zero_grad()   # clear gradients for next train
  loss.backward()         # backpropagation, compute gradients
  optimizer.step()
  # validate
  if i % 25 == 0:
    simple_net.training = False
    prediction = simple_net(torch.Tensor(small_simple_training).to(torch.device("cuda:0"))).cpu().detach().numpy()
    error = np.sum(np.square(prediction - small_labels.reshape((-1,1))))
    prediction = np.where(prediction < 0, -1, 1)
    correct = np.where((np.add(prediction, small_labels.reshape((-1,1))) == 0), 0, 1)
    print(np.sum(correct) / correct.size)
    print(error / correct.size)
    simple_net.training = True

0.52
1.1309068699233236
0.61
0.9668354926470651
0.67
0.8944593193754398
0.7
0.857221468783357
0.7
0.8475049580320463
0.73
0.8200684093262293
0.7
0.8062145113298239
0.7
0.7996008499824017
0.72
0.7927178471095452
0.73
0.7866140334066651
0.74
0.7772977586189762
0.68
0.7740252043945823
0.72
0.7716981291467852
0.72
0.7718559028713936
0.75
0.7676550763440284
0.75
0.7647415626756989
0.76
0.7615670250330824
0.78
0.7557361551452716
0.77
0.7502069859784767
0.76
0.7520574757149007
0.77
0.75215244610335
0.75
0.7585538486682378
0.77
0.7621819429284885
0.77
0.7640809567391915
0.78
0.7627375749565073
0.76
0.7609295617742743
0.75
0.7600839588343546
0.78
0.7480414587638401
0.76
0.742064941764694
0.8
0.7445502909941516
0.79
0.7409018367354228
0.77
0.7400347732180346
0.74
0.7327168484352687
0.76
0.7358642481359583
0.75
0.7310324669626717
0.77
0.7317645851430947
0.75
0.7337134378172465
0.75
0.7315747564537586
0.75
0.7357292140267652
0.74
0.7401725875330761


In [58]:
simple_net.training = False
prediction = simple_net(torch.Tensor(simple_val).to(torch.device("cuda:0"))).cpu().detach().numpy()
print(prediction[:20])
print(labels_val[:20])

RuntimeError: ignored

In [ ]:
print(data_set_simple)

In [13]:
from xgboost import XGBClassifier
eval_set = [(data_val, labels_val)]
eval_set_simple = [(simple_val, labels_val)]

In [23]:
# results = []
model_simple = XGBClassifier(n_estimators=10000, max_depth=10, learning_rate=.1, n_jobs=20, min_child_weight=1, gamma=.025, subsample=.8, colsample_bytree=1, tree_method = "gpu_hist", reg_alpha=1, reg_lambda=0)
model = XGBClassifier(n_estimators=10000, max_depth=10, learning_rate=.1, n_jobs=20, min_child_weight=1, gamma=.025, subsample=.8, colsample_bytree=1, tree_method = "gpu_hist", reg_alpha=1, reg_lambda=0)
model_simple.fit(simple_training, labels_training, eval_metric='mae', eval_set=eval_set_simple, early_stopping_rounds=100)
model.fit(data_training, labels_training, eval_metric='mae', eval_set=eval_set, early_stopping_rounds=100)
# results.append((em, np.sum(correct) / correct.size, np.sum(correct2) / correct2.size))

[0]	validation_0-mae:0.490684
Will train until validation_0-mae hasn't improved in 100 rounds.
[1]	validation_0-mae:0.481705
[2]	validation_0-mae:0.474138
[3]	validation_0-mae:0.467249
[4]	validation_0-mae:0.460851
[5]	validation_0-mae:0.454339
[6]	validation_0-mae:0.449007
[7]	validation_0-mae:0.444096
[8]	validation_0-mae:0.43982
[9]	validation_0-mae:0.435796
[10]	validation_0-mae:0.432492
[11]	validation_0-mae:0.428882
[12]	validation_0-mae:0.425603
[13]	validation_0-mae:0.422748
[14]	validation_0-mae:0.420276
[15]	validation_0-mae:0.418056
[16]	validation_0-mae:0.415875
[17]	validation_0-mae:0.414193
[18]	validation_0-mae:0.412243
[19]	validation_0-mae:0.410486
[20]	validation_0-mae:0.40894
[21]	validation_0-mae:0.407636
[22]	validation_0-mae:0.406902
[23]	validation_0-mae:0.405706
[24]	validation_0-mae:0.403959
[25]	validation_0-mae:0.402775
[26]	validation_0-mae:0.401918
[27]	validation_0-mae:0.401511
[28]	validation_0-mae:0.401035
[29]	validation_0-mae:0.400264
[30]	validation_0

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0.025,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=10000, n_jobs=20,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=1, reg_lambda=0, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, tree_method='gpu_hist', verbosity=1)

In [24]:
prediction = model.predict(data_val)
correct = np.where((prediction + labels_val == 1), 0, 1)
prediction2 = model_simple.predict(simple_val)
correct2 = np.where((prediction2 + labels_val == 1), 0, 1)
print((np.sum(correct) / correct.size, np.sum(correct2) / correct2.size))
# prediction = model.predict(data_test)
# correct = np.where((prediction + labels_test == 1), 0, 1)
# prediction2 = model_simple.predict(simple_test)
# correct2 = np.where((prediction2 + labels_test == 1), 0, 1)
# print((np.sum(correct) / correct.size, np.sum(correct2) / correct2.size))

(0.6671023965141613, 0.6196078431372549)


In [25]:
model.save_model('/ME/My Drive/Basketball_Data/0001.model')

In [18]:
prediction = model.predict_proba(data_val)
print(prediction[:20])
print(labels_val[:20])

[[9.9995148e-01 4.8494923e-05]
 [9.9995148e-01 4.8494923e-05]
 [9.9995148e-01 4.8494923e-05]
 [9.9995148e-01 4.8494923e-05]
 [9.9995148e-01 4.8494923e-05]
 [9.9995148e-01 4.8494923e-05]
 [9.9995148e-01 4.8494923e-05]
 [9.9995148e-01 4.8494923e-05]
 [9.9995148e-01 4.8494923e-05]
 [9.9995148e-01 4.8494923e-05]
 [9.9995148e-01 4.8494923e-05]
 [9.9995148e-01 4.8494923e-05]
 [9.9995148e-01 4.8494923e-05]
 [9.9995148e-01 4.8494923e-05]
 [9.9995148e-01 4.8494923e-05]
 [9.9995148e-01 4.8494923e-05]
 [9.9995148e-01 4.8494923e-05]
 [9.9995148e-01 4.8494923e-05]
 [9.9995148e-01 4.8494923e-05]
 [9.9995148e-01 4.8494923e-05]]
[ 16.  13.  10. -29.  -6.  -4.  -1.  21.   8.  -7.   6.   2.  -2.   4.
   9.  -4.   7. -14.  -2.  -2.]


In [109]:
prediction = model.predict(data_val)
error = np.sum(np.square(prediction - labels_val))
prediction = np.where(prediction < .5, 0, 1)
correct = np.where((prediction + labels_val == 1), 0, 1)
print(np.sum(correct) / correct.size)
print(error)

0.6561264822134387
174


In [26]:
from sklearn.neural_network import MLPClassifier

In [27]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(3125, 625, 125, 25, 5), random_state=1)
clf.fit(data_training, labels_training)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-f2496f86166f>", line 2, in <module>
    clf.fit(data_training, labels_training)
  File "/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py", line 995, in fit
    hasattr(self, "classes_")))
  File "/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py", line 375, in _fit
    intercept_grads, layer_units)
  File "/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py", line 469, in _fit_lbfgs
    args=(X, y, activations, deltas, coef_grads, intercept_grads))
  File "/usr/local/lib/python3.6/dist-packages/scipy/optimize/_minimize.py", line 610, in minimize
    callback=callback, **options)
  File "/usr/local/lib/python3.6/dist-packages/scipy/optimize/lbfgsb.py", line 34

KeyboardInterrupt: ignored

In [ ]:
prediction = clf.predict(data_val)
error = np.sum(np.square(prediction - labels_val))
prediction = np.where(prediction < .5, 0, 1)
correct = np.where((prediction + labels_val == 1), 0, 1)
print(np.sum(correct) / correct.size)
print(error)